In [1]:
%pip install ccrvam

Note: you may need to restart the kernel to use updated packages.


> Make sure to have ccrvam's latest version installed using `pip`. More information about the latest version can be found at https://pypi.org/project/ccrvam/

Run the following upgrade commands on your terminal if you are facing issues related to `pip` or `scipy`

```
# pip install --upgrade pip
# pip install --upgrade scipy
```

In [2]:
import numpy as np
from ccrvam import GenericCCRVAM, DataProcessor

# 2-Dimensional Case 

### Create Sample Contingency Table and Initialize the GenericCCRVAM

The `GenericCCRVAM` object can be initialized with a contingency table represented as a NumPy array. For a 2D contingency table:

- `axis=0`: First variable ($X_1$) with 5 categories
- `axis=1`: Second variable ($X_2$) with 3 categories

The axis indexing follows NumPy's convention, starting from the outermost dimension. The variables are ordered such that:

- $X_1$ corresponds to (rows)
- $X_2$ corresponds to (columns)

This ordering is important for calculating measures of regression association between two variables. And, later on in this `.ipynb`, you can see how we can conveniently mention the 1-indexed variable numbers to perform association measure calculation, regression, and prediction.

In [3]:
contingency_table = np.array([
    [0, 0, 20],
    [0, 10, 0],
    [20, 0, 0],
    [0, 10, 0],
    [0, 0, 20]
])

ccrvam_2d = GenericCCRVAM.from_contingency_table(contingency_table)
print(f"Shape of the inferred joint probability matrix P: {ccrvam_2d.P.shape}")
print(f"Joint Probability matrix P:\n{ccrvam_2d.P}")

Shape of the inferred joint probability matrix P: (5, 3)
Joint Probability matrix P:
[[0.    0.    0.25 ]
 [0.    0.125 0.   ]
 [0.25  0.    0.   ]
 [0.    0.125 0.   ]
 [0.    0.    0.25 ]]


### Calculating the Checkerboard Copula Scores (CCS) and their Variances

In [4]:
# Calculate and display scores for both variables
scores_X1 = ccrvam_2d.calculate_ccs(1)
scores_X2 = ccrvam_2d.calculate_ccs(2)

print("Checkerboard Copula Scores (CCS) for X1:")
print(scores_X1)
# Expected: [0.125, 0.3125, 0.5, 0.6875, 0.875]

print("\nCheckerboard Copula Scores (CCS) for X2:")
print(scores_X2)
# Expected: [0.125, 0.375, 0.75]

# Calculate and display variance of scores
variance_ccs_X1 = ccrvam_2d.calculate_variance_ccs(1)
variance_ccs_X2 = ccrvam_2d.calculate_variance_ccs(2)

print("\nVariance of Checkerboard Copula Scores (CCS) for X1:", variance_ccs_X1)
# Expected: 81/1024 = 0.0791015625
print("Variance of Checkerboard Copula Scores (CCS) for X2:", variance_ccs_X2)
# Expected: 9/128 = 0.0703125 

Checkerboard Copula Scores (CCS) for X1:
[np.float64(0.125), np.float64(0.3125), np.float64(0.5), np.float64(0.6875), np.float64(0.875)]

Checkerboard Copula Scores (CCS) for X2:
[np.float64(0.125), np.float64(0.375), np.float64(0.75)]

Variance of Checkerboard Copula Scores (CCS) for X1: 0.0791015625
Variance of Checkerboard Copula Scores (CCS) for X2: 0.0703125


### Category Prediction Using Checkerboard Copula Regression (CCR)

We can use the Checkerboard Copula Regression (CCR) for predicting the category of the ordinal response variable for each combination of categories of predictors. The `get_category_predictions_ccr()` method:

- Predicts the categories of response variable (to be passed in as `response` input argument) given predictor values (to be listed in `predictors` input argument)
- Returns predictions in an easy-to-read DataFrame format
- Supports custom variable names for better interpretation (Optional)

In [5]:
predictions_X1_to_X2 = ccrvam_2d.get_predictions_ccr(predictors=[1], response=2)
print("\nPredictions from X1 to X2:")
print(predictions_X1_to_X2)

# Example: Showcasing the use of custom variable names for the output
variable_to_name_dict = {1: "Income Bracket", 2: "Education Level"}
predictions_Education_to_Income = ccrvam_2d.get_predictions_ccr(predictors=[2], response=1, variable_names=variable_to_name_dict)
print("\nPredictions from Education Level to Income Bracket:")
print(predictions_Education_to_Income)


Predictions from X1 to X2:
   X1 Category  Predicted X2 Category
0            1                      3
1            2                      2
2            3                      1
3            4                      2
4            5                      3

Predictions from Education Level to Income Bracket:
   Education Level Category  Predicted Income Bracket Category
0                         1                                  3
1                         2                                  3
2                         3                                  3


### Calculating CCRAM & SCCRAM

The CCRAM (Checkerboard Copula Regression Association Measure) allows us to quantify the regression relationship between an ordinal response variable and a set of categorical predictors. In our example:

- Variables are 1-indexed: $X_1$ (rows) and $X_2$ (columns)
- X1 to X2 ($X_1 \rightarrow X_2$) measures how much X1 (to be listed in `predictors` input argument) explains the variation in X2 (to be passed in as `response` input argument) 
- Scaled version (SCCRAM) normalizes the CCRAM for properly assessing the magnitude of regression association  by taking into account the upperbound of the CCRAM. 

In [6]:
ccram_X1_to_X2 = ccrvam_2d.calculate_CCRAM(predictors=[1], response=2)
ccram_X2_to_X1 = ccrvam_2d.calculate_CCRAM(predictors=[2], response=1)
print(f"CCRAM X1 to X2: {ccram_X1_to_X2:.4f}")
print(f"CCRAM X2 to X1: {ccram_X2_to_X1:.4f}")

sccram_X1_to_X2 = ccrvam_2d.calculate_CCRAM(predictors=[1], response=2, scaled=True)
sccram_X2_to_X1 = ccrvam_2d.calculate_CCRAM(predictors=[2], response=1, scaled=True)
print(f"SCCRAM X1 to X2: {sccram_X1_to_X2:.4f}")
print(f"SCCRAM X2 to X1: {sccram_X2_to_X1:.4f}")

CCRAM X1 to X2: 0.8438
CCRAM X2 to X1: 0.0000
SCCRAM X1 to X2: 1.0000
SCCRAM X2 to X1: 0.0000


# 4-Dimensional Case (Real Data Analysis)

### Load the data in three types of form (case form, frequency form and table form) and Initialize the GenericCCRVAM

This example demonstrates how one can analyze relationships between multiple categorical variables in a clinical dataset of back pain treatments using the GenericCCRVAM object.

The dataset contains 4 categorical variables from a medical study:

| Variable | Description | Categories |
|----------|-------------|------------|
| X₁ | Length of Previous Attack | 1=Short, 2=Long |
| X₂ | Pain Change | 1=Better, 2=Same, 3=Worse |
| X₃ | Lordosis | 1=Absent/Decreasing, 2=Present/Increasing |
| X₄ | Back Pain Outcome | 1=Worse (W), 2=Same (S), 3=Slight Improvement (SI), 4=Moderate Improvement (MODI), 5=Marked Improvement (MARI), 6=Complete Relief (CR) |

1. Define variable names, category mappings, and shape tuple
2. Load case-form/freq-form data from file (from path provided as an argument) or type table-form with proper mappings into a contingency table
3. Initialize GenericCCRVAM object from contingency table
4. Extract and print:
   - Joint probability matrix P
   - Marginal probability distributions
   - Marginal cumulative distributions  
   - Conditional probability distributions

In [7]:
var_list_4d = ["x1", "x2", "x3", "pain"]
category_map_4d = {
    "pain": {
        "worse": 1,
        "same": 2,
        "slight.improvement": 3,
        "moderate.improvement": 4,
        "marked.improvement": 5,
        "complete.relief": 6
    },
}
data_shape = (2, 3, 2, 6)
contingency_table_4d = DataProcessor.load_data(
                        "./data/caseform.pain.txt",
                        data_form="case_form",
                        shape=data_shape,
                        var_list=var_list_4d,
                        category_map=category_map_4d,
                        named=True,
                        delimiter="\t"
                    )
print("Read contingency table from case form data:")
rda_ccrvam = GenericCCRVAM.from_contingency_table(contingency_table_4d)
print(f"Shape of the inferred joint probability matrix P: {rda_ccrvam.P.shape}")
print(f"Joint Probability matrix P:\n{rda_ccrvam.P}\n")

Read contingency table from case form data:
Shape of the inferred joint probability matrix P: (2, 3, 2, 6)
Joint Probability matrix P:
[[[[0.         0.00990099 0.         0.         0.01980198 0.03960396]
   [0.         0.         0.         0.00990099 0.02970297 0.        ]]

  [[0.         0.01980198 0.02970297 0.         0.05940594 0.03960396]
   [0.         0.00990099 0.         0.01980198 0.         0.00990099]]

  [[0.         0.         0.         0.         0.01980198 0.01980198]
   [0.         0.         0.00990099 0.00990099 0.02970297 0.        ]]]


 [[[0.         0.         0.02970297 0.         0.00990099 0.01980198]
   [0.         0.00990099 0.         0.         0.02970297 0.        ]]

  [[0.         0.02970297 0.03960396 0.04950495 0.05940594 0.01980198]
   [0.00990099 0.03960396 0.03960396 0.02970297 0.         0.00990099]]

  [[0.01980198 0.01980198 0.00990099 0.04950495 0.01980198 0.        ]
   [0.01980198 0.         0.01980198 0.02970297 0.         0.        ]]]

In [8]:
contingency_table_4d_from_freq = DataProcessor.load_data(
                        "./data/freqform.pain.txt",
                        data_form="frequency_form",
                        shape=data_shape,
                        var_list=var_list_4d,
                        category_map=category_map_4d,
                        named=True,
                        delimiter="\t"
                    )
print("Read contingency table from freq form data:")
rda_ccrvam_from_freq = GenericCCRVAM.from_contingency_table(contingency_table_4d_from_freq)
print(f"Shape of the inferred joint probability matrix P: {rda_ccrvam_from_freq.P.shape}")
print(f"Joint Probability matrix P:\n{rda_ccrvam_from_freq.P}\n")

Read contingency table from freq form data:
Shape of the inferred joint probability matrix P: (2, 3, 2, 6)
Joint Probability matrix P:
[[[[0.         0.00990099 0.         0.         0.01980198 0.03960396]
   [0.         0.         0.         0.00990099 0.02970297 0.        ]]

  [[0.         0.01980198 0.02970297 0.         0.05940594 0.03960396]
   [0.         0.00990099 0.         0.01980198 0.         0.00990099]]

  [[0.         0.         0.         0.         0.01980198 0.01980198]
   [0.         0.         0.00990099 0.00990099 0.02970297 0.        ]]]


 [[[0.         0.         0.02970297 0.         0.00990099 0.01980198]
   [0.         0.00990099 0.         0.         0.02970297 0.        ]]

  [[0.         0.02970297 0.03960396 0.04950495 0.05940594 0.01980198]
   [0.00990099 0.03960396 0.03960396 0.02970297 0.         0.00990099]]

  [[0.01980198 0.01980198 0.00990099 0.04950495 0.01980198 0.        ]
   [0.01980198 0.         0.01980198 0.02970297 0.         0.        ]]]

In [9]:
table = np.array([[[[0, 1, 0, 0, 2, 4],
                    [0, 0, 0, 1, 3, 0]],
                    [[0, 2, 3, 0, 6, 4],
                    [0, 1, 0, 2, 0, 1]],
                    [[0, 0, 0, 0, 2, 2],
                    [0, 0, 1, 1, 3, 0]]],
                    [[[0, 0, 3, 0, 1, 2],
                    [0, 1, 0, 0, 3, 0]],
                    [[0, 3, 4, 5, 6, 2],
                    [1, 4, 4, 3, 0, 1]],
                    [[2, 2, 1, 5, 2, 0],
                    [2, 0, 2, 3, 0, 0]]]])
contingency_table_4d_from_array = DataProcessor.load_data(
                        table,
                        data_form="table_form",
                        shape=data_shape,
                        var_list=var_list_4d,
                        category_map=category_map_4d,
                    )
print("Read contingency table from table form data:")
rda_ccrvam_from_array = GenericCCRVAM.from_contingency_table(contingency_table_4d_from_array)
print(f"Shape of the inferred joint probability matrix P: {rda_ccrvam_from_array.P.shape}")
print(f"Joint Probability matrix P:\n{rda_ccrvam_from_array.P}\n")

Read contingency table from table form data:
Shape of the inferred joint probability matrix P: (2, 3, 2, 6)
Joint Probability matrix P:
[[[[0.         0.00990099 0.         0.         0.01980198 0.03960396]
   [0.         0.         0.         0.00990099 0.02970297 0.        ]]

  [[0.         0.01980198 0.02970297 0.         0.05940594 0.03960396]
   [0.         0.00990099 0.         0.01980198 0.         0.00990099]]

  [[0.         0.         0.         0.         0.01980198 0.01980198]
   [0.         0.         0.00990099 0.00990099 0.02970297 0.        ]]]


 [[[0.         0.         0.02970297 0.         0.00990099 0.01980198]
   [0.         0.00990099 0.         0.         0.02970297 0.        ]]

  [[0.         0.02970297 0.03960396 0.04950495 0.05940594 0.01980198]
   [0.00990099 0.03960396 0.03960396 0.02970297 0.         0.00990099]]

  [[0.01980198 0.01980198 0.00990099 0.04950495 0.01980198 0.        ]
   [0.01980198 0.         0.01980198 0.02970297 0.         0.        ]]

In [10]:
print("Marginal pdfs:")
for idx, marginal_pdf in rda_ccrvam.marginal_pdfs.items():
    print(f"Case-form Marginal pdf for X{idx+1}: {marginal_pdf}")

print("\nMarginal cdfs:")
for idx, marginal_cdf in rda_ccrvam.marginal_cdfs.items():
    print(f"Case-form Marginal cdf for X{idx+1}: {marginal_cdf}")

Marginal pdfs:
Case-form Marginal pdf for X1: [0.38613861 0.61386139]
Case-form Marginal pdf for X2: [0.20792079 0.51485149 0.27722772]
Case-form Marginal pdf for X3: [0.63366337 0.36633663]
Case-form Marginal pdf for X4: [0.04950495 0.13861386 0.17821782 0.1980198  0.27722772 0.15841584]

Marginal cdfs:
Case-form Marginal cdf for X1: [0.         0.38613861 1.        ]
Case-form Marginal cdf for X2: [0.         0.20792079 0.72277228 1.        ]
Case-form Marginal cdf for X3: [0.         0.63366337 1.        ]
Case-form Marginal cdf for X4: [0.         0.04950495 0.18811881 0.36633663 0.56435644 0.84158416
 1.        ]


### Calculating Checkerboard Copula Scores (CCS) and their Variances

In [11]:
# Calculate and display scores for both variables
rda_scores_X1 = rda_ccrvam.calculate_ccs(1)
rda_scores_X2 = rda_ccrvam.calculate_ccs(2)
rda_scores_X3 = rda_ccrvam.calculate_ccs(3)
rda_scores_X4 = rda_ccrvam.calculate_ccs(4)

print("Checkerboard Copula Scores (CCS) for X1:")
print(rda_scores_X1)
print("\nCheckerboard Copula Scores (CCS) for X2:")
print(rda_scores_X2)
print("\nCheckerboard Copula Scores (CCS) for X3:")
print(rda_scores_X3)
print("\nCheckerboard Copula Scores (CCS) for X4:")
print(rda_scores_X4)

# Calculate and display variance of scores
rda_variance_ccs_X1 = rda_ccrvam.calculate_variance_ccs(1)
rda_variance_ccs_X2 = rda_ccrvam.calculate_variance_ccs(2)
rda_variance_ccs_X3 = rda_ccrvam.calculate_variance_ccs(3)
rda_variance_ccs_X4 = rda_ccrvam.calculate_variance_ccs(4)

print("\nVariance of Checkerboard Copula scores (CCS) for X1:", rda_variance_ccs_X1)
print("\nVariance of Checkerboard Copula scores (CCS) for X2:", rda_variance_ccs_X2)
print("\nVariance of Checkerboard Copula scores (CCS) for X3:", rda_variance_ccs_X3)
print("\nVariance of Checkerboard Copula scores (CCS) for X4:", rda_variance_ccs_X4)
# Expected 12 * (variance of Checkerboard Copula scores for X4): 0.07987568681385342*12 = 0.95850824176

Checkerboard Copula Scores (CCS) for X1:
[np.float64(0.19306930693069307), np.float64(0.693069306930693)]

Checkerboard Copula Scores (CCS) for X2:
[np.float64(0.10396039603960396), np.float64(0.4653465346534653), np.float64(0.8613861386138614)]

Checkerboard Copula Scores (CCS) for X3:
[np.float64(0.31683168316831684), np.float64(0.8168316831683169)]

Checkerboard Copula Scores (CCS) for X4:
[np.float64(0.024752475247524754), np.float64(0.1188118811881188), np.float64(0.27722772277227725), np.float64(0.4653465346534653), np.float64(0.7029702970297029), np.float64(0.9207920792079207)]

Variance of Checkerboard Copula scores (CCS) for X1: 0.059258896186648376

Variance of Checkerboard Copula scores (CCS) for X2: 0.06943601918274368

Variance of Checkerboard Copula scores (CCS) for X3: 0.058033526124889726

Variance of Checkerboard Copula scores (CCS) for X4: 0.07987568681385343


### Category Prediction Using Checkerboard Copula Regression (CCR)

We can use the Checkerboard Copula Regression (CCR) for predicting the category of the ordinal response variable for each combination of categories of predictors. The `get_category_predictions_ccr()` method:

- Predicts the categories of response variable (to be passed in as `response` input argument) given predictor values (to be listed in `predictors` input argument)
- Returns predictions in an easy-to-read DataFrame format
- Supports custom variable names for better interpretation (Optional)

In [12]:
rda_predictions_X1_X2_X3_to_X4 = rda_ccrvam.get_predictions_ccr(predictors=[1, 2, 3], response=4)
print("\nPredictions from X1, X2, X3 to Y = X4:")
print(rda_predictions_X1_X2_X3_to_X4)

rda_variable_to_name_dict = {1: "Length of Previous Attack", 2: "Pain Change", 3: "Lordosis", 4: "Back Pain"}
rda_predictions_X1_X2_X3_to_X4_named = rda_ccrvam.get_predictions_ccr(predictors=[1, 2, 3], response=4, variable_names=rda_variable_to_name_dict)
print("\nPredictions from Length of Previous Attack, Pain Change, Lordosis to Y = Back Pain:")
print(rda_predictions_X1_X2_X3_to_X4_named)


Predictions from X1, X2, X3 to Y = X4:
    X1 Category  X2 Category  X3 Category  Predicted X4 Category
0             1            1            1                      5
1             1            1            2                      5
2             1            2            1                      5
3             1            2            2                      4
4             1            3            1                      5
5             1            3            2                      5
6             2            1            1                      4
7             2            1            2                      4
8             2            2            1                      4
9             2            2            2                      3
10            2            3            1                      3
11            2            3            2                      3

Predictions from Length of Previous Attack, Pain Change, Lordosis to Y = Back Pain:
    Length of Previous Attack 

### Calculating CCRAM & SCCRAM

The CCRAM (Checkerboard Copula Regression Association Measure) allows us to quantify the regression relationship between an ordinal response variable and a set of categorical predictors. In our example:

- Variables are 1-indexed: (by default) $X_1$, $X_2$, $X_3$, $X_4$.
- (X1, X2, X3) to X4 ($(X_1, X_2, X_3) \rightarrow X_4$) measures how much (X1, X2, X3) (to be listed in `predictors` input argument) explain the variation in X4 (to be passed in as `response` input argument) 
- Scaled version (SCCRAM) normalizes the CCRAM for properly assessing the magnitude of regression association  by taking into account the upperbound of the CCRAM. 

In [13]:
rda_ccram_X1_X2_X3_to_X4 = rda_ccrvam.calculate_CCRAM(predictors=[1, 2, 3], response=4)
print(f"CCRAM from (X1, X2, X3) to X4: {rda_ccram_X1_X2_X3_to_X4:.4f}")

rda_sccram_X1_X2_X3_to_X4 = rda_ccrvam.calculate_CCRAM(predictors=[1, 2, 3], response=4, scaled=True)
print(f"SCCRAM from (X1, X2, X3) to X4: {rda_sccram_X1_X2_X3_to_X4:.4f}")

CCRAM from (X1, X2, X3) to X4: 0.2576
SCCRAM from (X1, X2, X3) to X4: 0.2687
